In [9]:

import sys
import numpy as np
import os
current_directory = os.getcwd()
relative_path1 = '../../build/env_simulator'
relative_path2 = '../../build/cilqr'
file_path1 = os.path.join(current_directory, relative_path1)
file_path2 = os.path.join(current_directory, relative_path2)
sys.path.append(file_path1)
sys.path.append(file_path2)
import pybind_env_simulator
relative_path3 = '../../env_simulator'
file_path3 = os.path.join(current_directory, relative_path3)
sys.path.append(file_path3)
relative_path4 = '../'
file_path4 = os.path.join(current_directory, relative_path4)
sys.path.append(file_path4)
relative_path5 = '../../build/dcp_tree'
file_path4 = os.path.join(current_directory, relative_path5)
sys.path.append(relative_path5)
relative_path6 = '../python_plot'
relative_path6 = os.path.join(current_directory, relative_path6)
sys.path.append(relative_path6)
import pybind_ilqr
import pybind_dcp_tree
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import WheelZoomTool
import json
from pprint import pprint
from collections import defaultdict
import itertools
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import LabelSet, ColumnDataSource
from python_plot import dcp_tree_plot
from bokeh.layouts import column, row
from sklearn.metrics import confusion_matrix
file_path = '/home/ws/pilotmdc_aarch64/senseauto-pilot-decision/common_math/algorithm/dcp_tree/data/'

In [10]:
import json
import glob
def read_name_and_real_action(json_files):
    real_action_list = []
    for json_file in json_files:
        with open(json_file, 'r') as file:
            data = json.load(file)
            name = data.get('name')
            real_action = data.get('real_action')
            # print(f"File: {json_file} - Name: {name}, Real Action: {real_action}")
            real_action_list.append((name, real_action))
    return real_action_list
# 使用glob模块找到当前目录下的所有JSON文件
json_files = glob.glob('*.json')
# print(json_files)
# 读取并打印每个文件的name和real_action
real_action_list = read_name_and_real_action(json_files)
pprint(real_action_list)


[('case2_crowded_3_lanes', '010'),
 ('case8_cur_left_right_3_cars', '010'),
 ('case7_cur_right_lane_2_cars', '100'),
 ('case5_only_have_behind_ego_cars', '010'),
 ('case1_empty_3_lanes', '010'),
 ('case4_two_lane_cars_too_close', '101'),
 ('case3_one_lane_slow_car', '101'),
 ('case6_cur_left_lane_2_cars', '001')]


In [11]:
dcp_tree_output = []
action_mapping = {
    0: '010',
    -1: '100',
    1: '001',
}
for case_name in json_files:
    print(f"************case name is: {case_name}************** ")
    file_name = file_path + case_name
    file = file_path + file_name
    dcp_tree = pybind_dcp_tree.DCPTree()
    root_node = pybind_dcp_tree.DCPTreeNodeData()
    ego_info = pybind_ilqr.IDMCarInfo(0.2, 16.6, 0.2, 5.0)
    root_node.obs_pos = dcp_tree_plot.read_json(file_name)
    root_node.ego_pos = ego_info
    dcp_tree.set_IDM_params(1.0, 3, 16.6)
    root_node.tree_trace.clear()
    dcp_tree.update_tree(root_node)
    dcp_tree.show()
    all_path = dcp_tree.get_all_path()
    min_cost_path_index = 0
    min_cost = all_path[0][-1][0]  # 获取第一条路径最后一个元素的成本

    for i in range(1, len(all_path)):
        current_path_last_cost = all_path[i][-1][0]
        if current_path_last_cost < min_cost:
            min_cost = current_path_last_cost
            min_cost_path_index = i

    # 打印最小成本路径信息
    print(f"Path with minimum last step cost (Path number {min_cost_path_index + 1}):")
    for step in all_path[min_cost_path_index]:
        print(f"Time: {step[1]}, Action: {step[2]}, Cost: {step[0]}")
    print(f"Minimum last step cost: {min_cost}")

    # 准备输出
    output = pybind_dcp_tree.DCPTreeOutput()
    if len(all_path[min_cost_path_index]) > 1:
        min_path_second_elem = all_path[min_cost_path_index][1]
        output.suggest_action_confidence = min_path_second_elem[0]
        output.suggest_action_relative_time = min_path_second_elem[1]
        # 注意: suggest_action需要转换为DCPActionDir的成员，这取决于min_path_second_elem[2]的具体值
        output.suggest_action = min_path_second_elem[2]  # 这里可能需要根据实际情况进行调整
    else:
        print("The minimum cost path does not have enough elements.")
    dcp_tree_output.append((case_name, output.suggest_action))
# pprint(dcp_tree_output)
dcp_tree_out_list = [(case[0], action_mapping[case[1]]) for case in dcp_tree_output]
pprint(dcp_tree_out_list)

************case name is: case2_crowded_3_lanes.json************** 
Path with minimum last step cost (Path number 1):
Time: 0.0, Action: DCPActionDir.LK, Cost: 0.0
Time: 1.0, Action: DCPActionDir.LK, Cost: -0.891566265060241
Time: 2.0, Action: DCPActionDir.LK, Cost: -1.6642102338531843
Time: 3.0, Action: DCPActionDir.LK, Cost: -2.463900812634964
Time: 4.0, Action: DCPActionDir.LK, Cost: -3.2903116290559975
Time: 5.0, Action: DCPActionDir.LK, Cost: -4.143138455633518
Time: 6.0, Action: DCPActionDir.LK, Cost: -5.022096863734871
Time: 7.0, Action: DCPActionDir.LK, Cost: -5.926920209874351
Time: 8.0, Action: DCPActionDir.LK, Cost: -6.857357896229743
Time: 9.0, Action: DCPActionDir.LK, Cost: -7.8131738592286855
Time: 10.0, Action: DCPActionDir.LK, Cost: -8.794145249201604
Time: 11.0, Action: DCPActionDir.LK, Cost: -9.80006127117692
Time: 12.0, Action: DCPActionDir.LK, Cost: -10.830722162426705
Minimum last step cost: -10.830722162426705
************case name is: case8_cur_left_right_3_cars.

In [12]:
# 生成混淆矩阵
actual = real_action_list
predicted =  dcp_tree_out_list
# 基于新的理解，我们只需要一个二元的混淆矩阵
# 0: 不变道, 1: 变道（包括左变道、右变道或左右都可变道）

# 初始化二元混淆矩阵：[不变道正确, 变道正确; 不变道错误（即预测变道但实际没变）, 变道错误（即预测不变道但实际变了）]
binary_confusion_matrix = [[0, 0], [0, 0]]

for actual, predicted in zip(actual, predicted):
    actual_behavior = actual[1]
    predicted_behavior = predicted[1]

    # 判断真实行为是变道还是不变道
    actual_change = actual_behavior in ["100", "001", "101"]
    predicted_change = predicted_behavior in ["100", "001"]

    if actual_change:
        # 真实变道
        if predicted_change:
            # 预测也变道，正确
            binary_confusion_matrix[1][1] += 1
        else:
            # 预测不变道，错误
            binary_confusion_matrix[1][0] += 1
    else:
        # 真实不变道
        if predicted_change:
            # 预测变道，错误
            binary_confusion_matrix[0][1] += 1
        else:
            # 预测也不变道，正确
            binary_confusion_matrix[0][0] += 1

pprint(binary_confusion_matrix)

#            预测不变道  预测变道
# 实际不变道     TN         FP 
# 实际变道       FN         TP
# 初始化两个列表，用于存储每种情况下的case名称
true_negative_cases = []  # 真负例：实际不变道，预测也不变道
false_positive_cases = []  # 假正例：实际不变道，但预测变道
false_negative_cases = []  # 假负例：实际变道，但预测不变道
true_positive_cases = []  # 真正例：实际变道，预测也变道

for actual, predicted in zip(real_action_list, dcp_tree_out_list):
    actual_case_name, actual_behavior = actual
    predicted_case_name, predicted_behavior = predicted

    # 判断真实行为是变道还是不变道
    actual_change = actual_behavior in ["100", "001", "101"]
    predicted_change = predicted_behavior in ["100", "001"]

    if actual_change:
        if predicted_change:
            true_positive_cases.append(actual_case_name)
        else:
            false_negative_cases.append(actual_case_name)
    else:
        if predicted_change:
            false_positive_cases.append(actual_case_name)
        else:
            true_negative_cases.append(actual_case_name)
# 真实不变道，预测不变道
print("True Negative 真实不变道，预测不变道 Cases:")
pprint(true_negative_cases)
# 真实不变道，预测变道
print("False Positive 真实不变道，预测变道 Cases:")
pprint(false_positive_cases)
# 真实变道，预测不变道
print("False Negative 真实变道，预测不变道 Cases:")
pprint(false_negative_cases)
# 真实变道，预测变道
print("True Positive 真实变道，预测变道 Cases:")
pprint(true_positive_cases)
print("*******************真实case***************************")
pprint(real_action_list)
print("*******************预测case***************************")
pprint(dcp_tree_out_list)


[[4, 0], [0, 4]]
True Negative 真实不变道，预测不变道 Cases:
['case2_crowded_3_lanes',
 'case8_cur_left_right_3_cars',
 'case5_only_have_behind_ego_cars',
 'case1_empty_3_lanes']
False Positive 真实不变道，预测变道 Cases:
[]
False Negative 真实变道，预测不变道 Cases:
[]
True Positive 真实变道，预测变道 Cases:
['case7_cur_right_lane_2_cars',
 'case4_two_lane_cars_too_close',
 'case3_one_lane_slow_car',
 'case6_cur_left_lane_2_cars']
*******************真实case***************************
[('case2_crowded_3_lanes', '010'),
 ('case8_cur_left_right_3_cars', '010'),
 ('case7_cur_right_lane_2_cars', '100'),
 ('case5_only_have_behind_ego_cars', '010'),
 ('case1_empty_3_lanes', '010'),
 ('case4_two_lane_cars_too_close', '101'),
 ('case3_one_lane_slow_car', '101'),
 ('case6_cur_left_lane_2_cars', '001')]
*******************预测case***************************
[('case2_crowded_3_lanes.json', '010'),
 ('case8_cur_left_right_3_cars.json', '010'),
 ('case7_cur_right_lane_2_cars.json', '100'),
 ('case5_only_have_behind_ego_cars.json', '010'),
 